In [198]:
import time
import threading
import numpy as np
import concurrent.futures

count = 0; inf = 1000
player, opponent = 'x', 'o'
lock = threading.Lock()

## General Functions

In [199]:
# Checks if playable tiles left on board
def isMovesLeft(board) :
    for i in range(3) :
        for j in range(3) :
            if (board[i][j] == '_') :
                return True
    return False

# Check if game ended(-10 or 10) or not(0)
def evaluate(b) :
    # Checking Rows for x or o victory.
    for row in range(3) :
        if (b[row][0] == b[row][1] and b[row][1] == b[row][2]) :
            if (b[row][0] == player) : return 10
            elif (b[row][0] == opponent) : return -10

    # Checking Columns for x or o victory.
    for col in range(3) :
        if (b[0][col] == b[1][col] and b[1][col] == b[2][col]) :
            if (b[0][col] == player) : return 10
            elif (b[0][col] == opponent) : return -10

    # Checking Diagonals for x or o victory.
    if (b[0][0] == b[1][1] and b[1][1] == b[2][2]) :
        if (b[0][0] == player) : return 10
        elif (b[0][0] == opponent) : return -10
    if (b[0][2] == b[1][1] and b[1][1] == b[2][0]) :
        if (b[0][2] == player) : return 10
        elif (b[0][2] == opponent) : return -10

    # If none have won then return 0
    return 0

# Get all possible moves given board state
def getmoves(board):
    moves = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == '_':
                moves.append((i,j))
    return moves

# Function to display the game board
def display(board):
    for line in board: print(line)
    print()

# Pre-defined boards for testing
def boards(i=None):
    b=[[[ 'x', 'o', 'x' ],
        [ '_', '_', '_' ],
        [ 'o', '_', '_' ]],
       [[ '_', 'o', '_' ],
        [ 'o', '_', 'x' ],
        [ '_', '_', '_' ]],
       [[ 'x', 'o', '_' ],
        [ '_', '_', '_' ],
        [ '_', '_', '_' ]]]
    if i: return b[i]
    else: return b

# Generates random board states
def rand_board(n=3):
    board=np.array([['_','_','_'],
                    ['_','_','_'],
                    ['_','_','_']])
    token = player
    # n random moves already played
    for _ in range(n):
        token = opponent if token == player else player
        while True:
            i = np.random.randint(0,3)
            j = np.random.randint(0,3)
            if board[i][j] == '_':
                board[i][j] = token
                break
    return board

## Simple Minimax

In [200]:
# Uses minimax to evaluate all possible ways a game can go
def minimax(board, depth, isMax) :
    globals()['count'] += 1         # Keeping track of number of explored nodes
    score = evaluate(board)
    if (score == 10 or score == -10) : return score    # if a player won the Game
    if (isMovesLeft(board) == False) : return 0        # If Game is drawn 

    if (isMax) :    # Maximizer
        best = -1000
        for i in range(3) :
            for j in range(3) :
                if (board[i][j]=='_') :     # For all playable tiles
                    board[i][j] = player
                    # Recursively call minimax for the next move
                    best = max( best, minimax(board, depth + 1, not isMax) )
                    board[i][j] = '_'
        return best
    else :          # Minimizer
        best = 1000
        for i in range(3) :
            for j in range(3) :
                if (board[i][j] == '_') :     # For all playable tiles
                    board[i][j] = opponent
                    # Recursively call minimax for the next move
                    best = min(best, minimax(board, depth + 1, not isMax))
                    board[i][j] = '_'
        return best
    
# Finds best possible move using minimax
def findBestSeq(board):
    globals()['count'] = 0
    bestMove = (-1, -1)
    bestVal = -inf
    
    for i in range(3) :
        for j in range(3) :
            if (board[i][j] == '_') :     # For all playable tiles
                board[i][j] = player
                # Calculate the minimax value for the current move
                moveVal = minimax(board, 0, False)
                board[i][j] = '_'
                if (moveVal > bestVal) :
                    bestMove = (i, j)
                    bestVal = moveVal
    return bestMove, bestVal            # Return best possible move with its score

## Sequential Alpha Beta 

In [201]:
# Alpha-Beta Pruning variant of Minimax
def abminimax(board, depth, isMax, alpha, beta) :
    globals()['count'] += 1         # Keeping track of number of explored nodes

    if (isMovesLeft(board) == False) : return 0        # If Game is drawn 
    score = evaluate(board)
    if (score == 10 or score == -10) : return score    # if a player won the Game

    if (isMax) :    # Maximizer
        value = -inf
        for i in range(3) :
            for j in range(3) :
                if (board[i][j] == '_') :     # For all playable tiles
                    board[i][j] = player
                    # Recursively call abminimax for the next move
                    value = max(value, abminimax(board, depth + 1, not isMax, alpha, beta) )
                    board[i][j] = '_'
                    alpha = max(alpha, value)   # Update Alpha Value
                    if alpha >= beta: break     # Prune branch
        return value
    else :          # Minimizer
        value = inf
        for i in range(3) :
            for j in range(3) :
                if (board[i][j] == '_') :     # For all playable tiles
                    board[i][j] = opponent
                    # Recursively call abminimax for the next move
                    value = min(value, abminimax(board, depth + 1, not isMax, alpha, beta))
                    board[i][j] = '_'
                    beta = min(value, beta)     # Update Beta Value
                    if beta <= alpha: break     # Prune branch
        return value

# Finds best possible move using alpha-beta pruning
def findBestAB(board) :
    globals()['count'] = 0
    bestMove = (-1, -1); alpha=-inf
    for i in range(3) :
        for j in range(3) :
            if (board[i][j] == '_') :     # For all playable tiles
                board[i][j] = player
                # Calculate the abminimax value for the current move
                value = abminimax(board, 0, False, alpha, inf)
                # Update alpha and best move if better move found
                if value > alpha: alpha = value; bestMove = (i, j)
                board[i][j] = '_'
    return bestMove, alpha              # Return best possible move with its score

In [202]:
# Testing code for Minimax With/Without Alpha-Beta
averages = [0,0]
zboards = boards()  # Use pre-defined testing boards
for idx,board in enumerate(zboards):
    print("Testcase:",idx+1)
    display(board)
    
    print("Without Alpha-Beta")
    bestMove,val = findBestSeq(board.copy())
    averages[0] += count
    print(f"   Optimal Move : {bestMove}, value: {val}")
    print("   Nodes Explored: ", count)
    #print("------------------------")

    print("With Alpha-Beta")
    bestMove,val = findBestAB(board.copy())
    averages[1] += count
    print(f"   Optimal Move : {bestMove}, value: {val}")
    print("   Nodes Explored: ", count)
    print("---------------------------------------")

print("\nWithout Alpha-Beta Average :", round(averages[0]/len(zboards)))
print("With Alpha-Beta Average :", round(averages[1]/len(zboards)))

Testcase: 1
['x', 'o', 'x']
['_', '_', '_']
['o', '_', '_']

Without Alpha-Beta
   Optimal Move : (2, 2), value: 10
   Nodes Explored:  257
With Alpha-Beta
   Optimal Move : (2, 2), value: 10
   Nodes Explored:  183
---------------------------------------
Testcase: 2
['_', 'o', '_']
['o', '_', 'x']
['_', '_', '_']

Without Alpha-Beta
   Optimal Move : (0, 0), value: 0
   Nodes Explored:  1420
With Alpha-Beta
   Optimal Move : (0, 0), value: 0
   Nodes Explored:  673
---------------------------------------
Testcase: 3
['x', 'o', '_']
['_', '_', '_']
['_', '_', '_']

Without Alpha-Beta
   Optimal Move : (1, 0), value: 10
   Nodes Explored:  8231
With Alpha-Beta
   Optimal Move : (1, 0), value: 10
   Nodes Explored:  1707
---------------------------------------

Without Alpha-Beta Average : 3303
With Alpha-Beta Average : 854


## Parallel Alpha Beta

In [211]:
# Parallelized Alpha-Beta Pruning variant of Minimax
def abpminimax(board, depth, isMax, alpha, beta, id) :
    globals()['counts'][id] += 1         # Keeping track of number of explored nodes

    if (isMovesLeft(board) == False) : return 0        # If Game is drawn 
    score = evaluate(board)
    if (score == 10 or score == -10) : return score    # if a player won the Game

    if (isMax) :    # Maximizer
        value = -inf
        for i in range(3) :
            for j in range(3) :
                if (board[i][j] == '_') :     # For all playable tiles
                    board[i][j] = player
                    # Recursively call abpminimax for the next move
                    value = max(value, abpminimax(board, depth + 1, not isMax, alpha, beta, id))
                    board[i][j] = '_'
                    alpha = max(alpha, value)   # Update Alpha Value
                    if alpha >= beta: break     # Prune branch
    else :          # Minimizer
        value = inf
        for i in range(3) :
            for j in range(3) :
                if (board[i][j] == '_') :     # For all playable tiles
                    board[i][j] = opponent
                    # Recursively call abpminimax for the next move
                    value = min(value, abpminimax(board, depth + 1, not isMax, alpha, beta, id))
                    board[i][j] = '_'
                    beta = min(value, beta)     # Update Beta Value
                    if beta <= alpha: break     # Prune branch
    return value

# Finds best possible move using parallelized alpha-beta pruning
def parallelAB(thread_data):
    board, i, j, alpha, id = thread_data
    global lock, threadres

    board[i][j] = player    # Play Assigned Move
    # Calculate the abpminimax value for the current move
    value = abpminimax(board, 0, False, alpha, inf, id)
    board[i][j] = '_'
    with lock:  # If current move is better, return move with value using a queue
        if value > alpha: threadres.append([(i,j), value])
        # else: threadres.append(((-1,-1), -inf))

# Parallel execution of the Alpha-Beta Pruning algorithm
def findBestABP(board):
    global count, threadres, counts
    count = 0
    res = ((-1,-1), -inf)
    moves = getmoves(board)
    boardc = np.array(board).tolist()       # Copy Board for second thread
    
    # Use ThreadPoolExecutor with 2 max_threads to parallelize the Alpha-Beta Pruning
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        # Process iterates in pairs to submit tasks for parallel execution
        for idx in range(0, len(moves), 2):
            i, j = moves[idx]
            if idx == len(moves) - 1:       # Execute last odd move sequentially
                parallelAB((board, i, j, res[1], 0))
            else:
                k, l = moves[idx + 1]
                # Submit two moves as tasks to the executor for parallel execution
                future1 = executor.submit(parallelAB, (board, i, j, res[1], 0))
                future2 = executor.submit(parallelAB, (boardc, k, l, res[1], 1))
                # Wait for the tasks to complete
                future1.result(); future2.result()
        
            count += sum(counts)            # Aggregate total nodes explored by both threads
            counts[0] = 0; counts[1] = 0        # Reset counts for next iteration
            # Find the best move and score from the results
            res = max(threadres, key=lambda x: x[1])
            threadres = [res]       # Reset the queue for next iteration
    return res                      # Return the best move and its score

In [602]:
# Testing code for Minimax Serial/Parallel Alpha-Beta
averages = [0,0]; time_avgs = [0,0]
counts = np.array([0,0])
zboards = boards()
threadres = []

for idx,board in enumerate(zboards):
    print("Testcase:",idx+1)
    display(board)
    
    print("Serial Alpha-Beta")
    stime = time.time()
    bestMove,val = findBestAB(board.copy())
    etime = time.time()
    averages[0] += count; time_avgs[0] += etime-stime
    print(f"   Optimal Move : {bestMove}, value: {val}")
    print("   Nodes Explored :", count)
    print(f"   Time Taken : {etime-stime:.5f} s")

    print("Parallel Alpha-Beta")
    stime = time.time()
    bestMove,val = findBestABP(board.copy())
    etime = time.time()
    averages[1] += count; time_avgs[1] += etime-stime
    print(f"   Optimal Move : {bestMove}, value: {val}")
    print("   Nodes Explored :", count)
    print(f"   Time Taken : {etime-stime:.5f} s")
    print("---------------------------------------")
    
print(f"\nSerial Alpha-Beta Average : {averages[0]/len(zboards):.0f}, {time_avgs[0]/len(zboards)*1000:.5f} ms")
print(f"Parallel Alpha-Beta Average : {averages[1]/len(zboards):.0f}, {time_avgs[1]/len(zboards)*1000:.5f} ms")

Testcase: 1
['x', 'o', 'x']
['_', '_', '_']
['o', '_', '_']

Serial Alpha-Beta
   Optimal Move : (2, 2), value: 10
   Nodes Explored : 183
   Time Taken : 0.01029 s
Parallel Alpha-Beta
   Optimal Move : (2, 2), value: 10
   Nodes Explored : 183
   Time Taken : 0.00057 s
---------------------------------------
Testcase: 2
['_', 'o', '_']
['o', '_', 'x']
['_', '_', '_']

Serial Alpha-Beta
   Optimal Move : (0, 0), value: 0
   Nodes Explored : 673
   Time Taken : 0.00758 s
Parallel Alpha-Beta
   Optimal Move : (2, 2), value: 10
   Nodes Explored : 606
   Time Taken : 0.00000 s
---------------------------------------
Testcase: 3
['x', 'o', '_']
['_', '_', '_']
['_', '_', '_']

Serial Alpha-Beta
   Optimal Move : (1, 0), value: 10
   Nodes Explored : 1707
   Time Taken : 0.01637 s
Parallel Alpha-Beta
   Optimal Move : (2, 2), value: 10
   Nodes Explored : 1787
   Time Taken : 0.01851 s
---------------------------------------

Serial Alpha-Beta Average : 854, 11.41111 ms
Parallel Alpha-Beta 

## Heuristic Alpha-Beta

In [603]:
# Initialize score table for Heuristic Calculation
score_table=np.zeros((4,4),dtype=int)
for index in range(0,4):
    score_table[index,0]=10**index
    score_table[0,index]=-10**index

# Score of a line where line is 3 consecutive cells (row, col, diag)
def eval_line(line, token=player):
    opp = opponent if token == player else player
    p_count = list(line).count(token)
    o_count = list(line).count(opp)
    return score_table[p_count,o_count]

# Calculate heuristic value for a board state
def heuristic(board,token=player):
    grid = np.array(board); h_val = 0
    for i in range(3):
        h_val += eval_line(grid[i],token)
        h_val += eval_line(grid[:,i],token)
    h_val += eval_line(grid.diagonal(),token)
    ld = [grid[0,2],grid[1,1],grid[2,0]]
    h_val += eval_line(ld, token)
    return h_val

# Check if a player won
def checkWin(board):
    score = evaluate(board)
    if score == -10: return opponent
    elif score == 10: return player
    else: return None

# Remove unusable moves for AI
def clear_unusable(moves,move):
    try: moves.remove(move)
    except: pass

# Calculate scores for each move
def moves_score(board):
    grid = np.array(board)
    moves = getmoves(board)

    scores = np.zeros(len(moves),dtype=int)
    for i,move in enumerate(moves):
        grid[move] = player
        scores[i] = heuristic(grid)
        grid[move] = '_'
    # Return moves with their heuristic scores
    return np.column_stack((moves, scores))

In [609]:
# Heuristic based Alpha-Beta Pruning variant of Minimax
def abhminimax(board, depth, isMax, alpha, beta) :
    globals()['count'] += 1         # Keeping track of number of explored nodes

    if depth == 0 or checkWin(board) is not None:        # If Game has ended(Draw or Win)
        return evaluate(board)
    moves = moves_score(board)      # Get all possible moves

    if (isMax) :    # Maximizer
        value = -inf
        # Sort Moves based on heuristic score to set exploration order
        move_score = sorted(moves, key=lambda x:x[2],reverse=True)  # Descending
        for i,j,_ in move_score:
            board[i][j] = player
            # Recursively call abhminimax for the next move
            value = max(value, abhminimax(board, depth - 1, not isMax, alpha, beta) )
            board[i][j] = '_'
            alpha = max(alpha, value)   # Update Alpha Value
            if alpha >= beta: break     # Prune branch
    else :          # Minimizer
        value = inf
        # Sort Moves based on heuristic score to set exploration order
        move_score = sorted(moves, key=lambda x:x[2],reverse=False)  # Ascending
        for i,j,_ in move_score:
            board[i][j] = opponent
            # Recursively call abhminimax for the next move
            value = min(value, abhminimax(board, depth - 1, not isMax, alpha, beta))
            board[i][j] = '_'
            beta = min(value, beta)     # Update Beta Value
            if beta <= alpha: break     # Prune branch
    return value

# Finds best possible move using heuristic alpha-beta pruning
def findBestABH(board) :
    globals()['count'] = 0
    bestMove = (-1, -1); alpha = -inf
    moves = moves_score(board); depth = len(moves)
    # Sort Moves based on heuristic score to set exploration order
    moves = sorted(moves, key=lambda x:x[2],reverse=True)

    for i,j,_ in moves:
        board[i][j] = player
        # Calculate the abpminimax value for the current move
        value = abhminimax(board, depth-1, False, alpha, inf)
        board[i][j] = '_'
        if value > alpha:
            alpha = value
            bestMove = (i, j)
    return bestMove, alpha             # Return the best move and its score

In [610]:
# Testing code for Minimax Simple/Heuristic Alpha-Beta
averages = [0,0]
zboards = boards()
for idx,board in enumerate(zboards):
    print("Testcase:",idx+1)
    display(board)

    print("Simple Alpha-Beta")
    bestMove,val = findBestAB(board)
    averages[0] += count
    print(f"   Optimal Move : {bestMove}, value: {val}")
    print("   Nodes Explored: ", count)
    
    print("Heuristic Alpha-Beta")
    bestMove,val = findBestABH(board)
    averages[1] += count
    print(f"   Optimal Move : {bestMove}, value: {val}")
    print("   Nodes Explored: ", count)
    print("---------------------------------------")

print("\nSimple Alpha-Beta Average :", round(averages[0]/len(zboards)))
print("Heuristic Alpha-Beta Average :", round(averages[1]/len(zboards)))

Testcase: 1
['x', 'o', 'x']
['_', '_', '_']
['o', '_', '_']

Simple Alpha-Beta
   Optimal Move : (2, 2), value: 10
   Nodes Explored:  183
Heuristic Alpha-Beta
   Optimal Move : (2, 2), value: 10
   Nodes Explored:  51
---------------------------------------
Testcase: 2
['_', 'o', '_']
['o', '_', 'x']
['_', '_', '_']

Simple Alpha-Beta
   Optimal Move : (0, 0), value: 0
   Nodes Explored:  673
Heuristic Alpha-Beta
   Optimal Move : (2, 0), value: 0
   Nodes Explored:  307
---------------------------------------
Testcase: 3
['x', 'o', '_']
['_', '_', '_']
['_', '_', '_']

Simple Alpha-Beta
   Optimal Move : (1, 0), value: 10
   Nodes Explored:  1707
Heuristic Alpha-Beta
   Optimal Move : (1, 1), value: 10
   Nodes Explored:  261
---------------------------------------

Simple Alpha-Beta Average : 854
Heuristic Alpha-Beta Average : 206


_________________________________________________________

### Optimizations of Parallel Alpha-Beta (Minor Gains)

In [606]:
# # Version 1: Without Thread Pool from 0
# def abpminimax(board, depth, isMax, alpha, beta, id) :
#     globals()['counts'][id] += 1

#     if (isMovesLeft(board) == False) : return 0
#     score = evaluate(board)
#     if (score == 10 or score == -10) : return score

#     if (isMax) :
#         value = -inf
#         for i in range(3) :
#             for j in range(3) :
#                 if (board[i][j]=='_') :
#                     board[i][j] = player
#                     value = max(value, abpminimax(board, depth + 1, not isMax, alpha, beta, id))
#                     board[i][j] = '_'
#                     alpha = max(alpha, value)
#                     if alpha >= beta: break
#     else :
#         value = inf
#         for i in range(3) :
#             for j in range(3) :
#                 if (board[i][j] == '_') :
#                     board[i][j] = opponent
#                     value = min(value, abpminimax(board, depth + 1, not isMax, alpha, beta, id))
#                     board[i][j] = '_'
#                     beta = min(value, beta)
#                     if beta <= alpha: break
#     return value

# def findBestABP(board, i, j, alpha, id):
#     global lock
#     board[i][j] = player
#     value = abpminimax(board, 0, False, alpha, inf, id)
#     board[i][j] = '_'
#     with lock:
#         if value > alpha: threadres.append([(i,j), value])
#         else: threadres.append([(-1,-1), -inf])

# def parallel(board):
#     globals()['count'] = 0
#     res = [(-1,-1), -inf]
#     moves = getmoves(board)
#     boardc = np.array(board).tolist()

#     for idx in range(0,len(moves),2):
#         i,j = moves[idx]
#         if idx == len(moves) - 1: 
#             findBestABP(board, i,j, res[1], 0)
#         else:
#             k,l = moves[idx+1]
#             thread1 = threading.Thread(target=findBestABP,args=[board,i,j,res[1],0])
#             thread2 = threading.Thread(target=findBestABP,args=[boardc,k,l,res[1],1])
#             thread1.start(); thread2.start()
#             thread1.join(); thread2.join()

#         globals()['count'] += sum(counts)
#         counts[0] = 0; counts[1] = 0 
#         threadres.append(res)
#         res = max(threadres, key =lambda x: x[1])
#         threadres.clear()
#     return res

# # Serial/Parallel Alpha Beta
# averages = [0,0]
# time_avgs = [0,0]
# counts = [0,0]

# for b in boards():
#     %timeit findBestAB(b.copy())
#     %timeit parallel(b.copy())

In [607]:
# # Version 2: Without Thread Pool from 2
# def abpminimax(board, depth, isMax, alpha, beta, id) :
#     globals()['counts'][id] += 1

#     if (isMovesLeft(board) == False) : return 0
#     score = evaluate(board)
#     if (score == 10 or score == -10) : return score

#     if (isMax) :
#         value = -inf
#         for i in range(3) :
#             for j in range(3) :
#                 if (board[i][j]=='_') :
#                     board[i][j] = player
#                     value = max(value, abpminimax(board, depth + 1, not isMax, alpha, beta, id))
#                     board[i][j] = '_'
#                     alpha = max(alpha, value)
#                     if alpha >= beta: break
#     else :
#         value = inf
#         for i in range(3) :
#             for j in range(3) :
#                 if (board[i][j] == '_') :
#                     board[i][j] = opponent
#                     value = min(value, abpminimax(board, depth + 1, not isMax, alpha, beta, id))
#                     board[i][j] = '_'
#                     beta = min(value, beta)
#                     if beta <= alpha: break
#     return value

# def findBestABP1(board, i, j, alpha, id):
#     global lock
#     board[i][j] = player
#     value = abpminimax(board, 0, False, alpha, inf, id)
#     board[i][j] = '_'
#     with lock:
#         if value > alpha: threadres.append([(i,j), value])
#         else: threadres.append([(-1,-1), -inf])

# def parallel(board):
#     global count, counts
#     count = 0
#     res = [(-1,-1), -inf]
#     moves = getmoves(board)
#     boardc = np.array(board).tolist()
#     for id,(i,j) in enumerate(moves):
#         findBestABP1(board, i, j, res[1], id)
#         count += counts[id]
#         res = max(threadres[id], res, key=lambda x: x[1])
#         if id == 1: break

#     threadres.clear()
#     counts[0] = 0; counts[1] = 0
#     for idx in range(2,len(moves),2):
#         i,j = moves[idx]
#         if idx == len(moves) - 1: 
#             findBestABP(board, i,j, res[1], 0)
#         else:
#             k,l = moves[idx+1]
#             thread1 = threading.Thread(target=findBestABP,args=[board,i,j,res[1],0])
#             thread2 = threading.Thread(target=findBestABP,args=[boardc,k,l,res[1],1])
#             thread1.start(); thread2.start()
#             thread1.join(); thread2.join()

#         count += sum(counts)
#         counts[0] = 0; counts[1] = 0 
#         threadres.append(res)
#         res = max(threadres, key =lambda x: x[1])
#         threadres.clear()
#     return res

# # Serial/Parallel Alpha Beta
# averages = [0,0]
# time_avgs = [0,0]
# counts = [0,0]
# for b in boards():
#     %timeit findBestAB(b.copy())
#     %timeit parallel(b.copy())

In [608]:
# # Version 3: Thread Pool from 2
# def findBestABP(thread_args):
#     board, i, j, alpha, id = thread_args
#     global lock
#     board[i][j] = player
#     value = abpminimax(board, 0, False, alpha, inf, id)
#     board[i][j] = '_'
#     with lock:
#         if value > alpha: threadres.append([(i,j), value])
#         else: threadres.append([(-1,-1), -inf])

# def parallel(board):
#     global count
#     count = 0
#     res = [(-1,-1), -inf]
#     moves = getmoves(board)
#     boardc = np.array(board).tolist()

#     for id,(i,j) in enumerate(moves):
#         findBestABP((board, i, j, res[1], id))
#         count += counts[id]
#         res = max(threadres[id], res, key=lambda x: x[1])
#         #print(f"({i} {j}): {counts[id]}")
#         if id == 1: break

#     threadres.clear()
#     counts[:] = 0

#     with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
#         for idx in range(2, len(moves), 2):
#             i, j = moves[idx]
#             if idx == len(moves) - 1:
#                 findBestABP((board, i, j, res[1], 0))
#             #    print(f"s({i} {j}): {counts[0]}")
#             else:
#                 k, l = moves[idx + 1]
#                 future1 = executor.submit(findBestABP, (board, i, j, res[1], 0))
#                 future2 = executor.submit(findBestABP, (boardc, k, l, res[1], 1))
#                 future1.result(); future2.result()
                
#             #   print(f"({i} {j}): {counts[0]}")
#             #   print(f"({k} {l}): {counts[1]}")
            
#             count += sum(counts)
#             counts[0] = 0; counts[1] = 0
#             bestSoFar = max(threadres, key=lambda x: x[1])
#             res = max(bestSoFar, res, key=lambda x: x[1])
#             threadres.clear()
#     return res


# # Serial/Parallel Alpha Beta
# averages = [0,0]
# time_avgs = [0,0]
# counts = np.array([0,0])

# for b in boards():
#     %timeit findBestAB(b.copy())
#     %timeit parallel(b.copy())